In [63]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt


df = pd.read_csv(u'../data-mining/data/iris.txt',sep=' ')

In [64]:
A = np.hstack([
        np.matrix(df.sl).T, 
        np.matrix(df.sw).T, 
        np.matrix(df.pl).T, 
        np.matrix(df.pw).T])
A[:5] # sample view

matrix([[ 5.1,  3.5,  1.4,  0.2],
        [ 4.9,  3. ,  1.4,  0.2],
        [ 4.7,  3.2,  1.3,  0.2],
        [ 4.6,  3.1,  1.5,  0.2],
        [ 5. ,  3.6,  1.4,  0.2]])

In [65]:
c = np.matrix(df.c).T
x_test = A[1,:] + 5*np.random.randn(1,4)
x_test

matrix([[ 12.09592687,   1.46990224,  -7.95584798,  -3.05405899]])

In [66]:
# Use a heap to store the smallest items

import heapq as hp

# Define an object and overload custom comparison operators
class tup:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val,self.idx)
    

In [67]:
def kNearestNeighbour(A,c,test,k): 
    heap = []
    N = A.shape[0]   
     ## Fill in the heap with dummy nodes
    for i in range(k):
        hp.heappush(heap, tup(np.inf, -1))

    for i in range(N):
        e = A[i,:]-test
        e = e.reshape((4,1))
        tp = tup(float(e.T*e), i)
        if tp <= heap[0]:
            hp.heapreplace(heap, tp)
    
    categories = []
    for j in range(len(heap)):
        h = hp.heappop(heap)
        categories.append(int(c[h.idx]))
    return max(set(categories), key=categories.count)

In [68]:
result=kNearestNeighbour(A,c,x_test,15)
result

1

In [71]:
# Recall and Precision Part
def calculateMeasures(k, A, category):
    false_positive = 0
    false_negative = 0
    true_positive = 0
    true_negative = 0
    
    
    for i in range(len(c)):
        result = kNearestNeighbour(A,c, A[i,:],k);
        if(result == int(c[i]) and int(c[i]) == category):
            true_positive += 1
        if(result == int(c[i]) and int(c[i]) != category):
            true_negative += 1
        if(result != int(c[i]) and int(c[i]) == category):
            false_negative += 1
        if(result != int(c[i]) and result == category):
            false_positive += 1
            
    precision = true_positive / (true_positive + false_positive) * 100
    recall = true_positive / (true_positive + false_negative) * 100
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative) * 100
        
    return (precision, recall, accuracy)

In [73]:
precision, recall, accuracy=calculateMeasures(15, A, 2)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

precision: 98.0
recall: 98.0
accuracy: 98.66666666666667
